<h1><center>Keen Problem</center></h1>

Made By :

* Abhishek Varghese (160010018)

## Objective

The aim of this assignment is to schedule the timetable of classes in IIT Goa such that the timetable satisfies all the constraints mentioned below. 

## Constraints

The assignment of the classes must satisfy the following constraints:

1. No two courses should be assigned the same room at the same time.

1. A course should not be assigned more than 1 rooms at the same time.

1. Each course should be assigned the type of classroom required for it (mentioned in the input file) and not any other type of room.

1. A professor should not be given two or more classes at the same time.

1. If two professors teach a particular course, then they both should be present in the lecture of that course.

1. A batch of students should not be given two or more courses at the same time.

1. All the lecture hours should fall in the institute timings (mentioned in the input file).

1. Start time And End time should be multiples of 30 min

1. Each course should be given the required number of time slots (mentioned in the input file).

1. Preference Constraints (from input file).

## Working

The approach to solve the classroom scheduling problem is to define appropriate propositions and write all the constraints. We pass all these constraints in a SAT solver. The SAT solver returns a satisfying assignment of the propositions if satisfiable. For this problem, the SAT solver used is Z3.

### Sample Input

The input is given in the form of a json file. The json file can be a dictionary or a list of lists which contains the input.

In [16]:
{
	"Room Types": ["small", "big", "chemistry", "physics"],
	"Institute time": [
		[8.50, 12.50],
		[14.00, 17.00]
	],
	"Classrooms": [
		["T1", "small"],
		["LH1", "big"],
		["physics_lab", "physics"],
		["T2", "small"],
		["LH2", "big"]
	],
	"Courses": [
    	["cs101", "physics", [3],
			["Biswas"], 1
		],
		["cs201", "small", [1.5, 1.5],
			["Amal", "Clint"], 2
		],
		["cs25", "big", [1, 1, 1],
			["Sharad"], 3
		],
		["cs228", "small", [1, 1, 1],
			["Sreejith"], 3
		]
	],
	"Preference prof not prefer": [
	["Amal", [13.0, 19.0] ],
	["Sharad", [38.0, 42.0] ],
	["Biswas", [0,70]]
	],
	"Preference batch not prefer": [
	[3, [62.0, 70.0]],
	[2, [45.0, 55.0]]
	]
};

### Model

Any problem's efficiency and speed depends upon how we model the problem. To do so we must first understand the given problem and look at it from various different perspective. Once the problem is understood we try to optimize the model as much as possible from the observations made of the problem. In the given problem following were observed :

* There was no need for a seperate proposition for assigning faculty as the faculty given in the input are the one's who would be teaching. Hence as a result any constraint on faculty would be similar to the same constraint on the courses.
* We dont need a constraint for every type of possible rooms for a given course. As the room type is given (small, large, pysics, etc) and as the course should run in only one of the rooms of the same type, we only have to take these rooms into account for each course instead of making propositions of all the rooms.(E.g. For the input given above for a class of cs201 the propositions of the room is cs201_t1, cs201_t2. And the constraint will be exactly one of these is true. )
* We need a model which can represent any general institute time. For this we represented the Institute time in a Number line from 0 - 70. A unit of 1 in this number line corresponds to half an hour. i.e. If a class is assigned time from 2 - 4 it means that the class will occur on Monday from 9 AM to 10 AM provided the start time is 8:30. By this we can use intiger propositions from z3 to represent time. This will ensure that all time will fall in multiples of 30 min.


Because of the above model we have clubbed many constraints into propositions itself. Hence the number of constraints and propositions have been drastically reduced, and hence the code will take less time as compared to one where such modelling was not done. (Avg. execution time of our code : 10 sec) 

## Code

### Headers and Libraries

In [17]:
from __future__ import division
import json
from z3 import *
from beautifultable import BeautifulTable


### Functions 

* A list and solver is given as input. This function ensures that only one of the boolean variable in the list is set to True and others are set to false. It does this by inserting following expression into the solver : Sum(over all variables k) k* Product(over all variables j not equal to k) !j

In [18]:
def exactly_one(prop_list,solver) :
    cnf = []
    for i in range(len(prop_list)) :
        temp = []
        for j in range(len(prop_list)):
            if j == i :
                temp.append(prop_list[j])
            else :
                temp.append(Not(prop_list[j]))
        cnf.append(And(temp))
    solver.add(Or(cnf))

* Four Integers are given. The function adds the constraint that the intersection of this should be phi

In [19]:
def intersection_phi(a_start,a_end,b_start,b_end,solver) :
    one = And([a_start < b_start, a_start < b_end, a_end < b_start, a_end < b_end ])
    two = And([a_start > b_start, a_start > b_end, a_end > b_start, a_end > b_end ])
    solver.add(Or(one,two))

* Similar to Above function but instead of directly adding the constraint to solver it is returning the same. The start and End of same type has been merged into a list which can be seen as the parameters.

In [20]:
def intersection_phi_return( class_a,class_b) :
    one = And([class_a[0] < class_b[0], class_a[0] < class_b[1], class_a[1] < class_b[0], class_a[1] < class_b[1] ])
    two = And([class_a[0] > class_b[0], class_a[0] > class_b[1], class_a[1] > class_b[0], class_a[1] > class_b[1] ])
    return (Or(one,two))

### Data parsing from json file

In [21]:
inpfile = ""

with open(inpfile) as readfile :
    a = readfile.read()

a = a.strip()
data = json.loads(a)

The input from the json file is stored in 'data'.

### Declarartion of variables and propositions & Preprocessing

In [ ]:
import numpy as np

column_indices = ["a",'b','c','d','e','f']
row_indices = [ str(i) for i in range(6,0,-1)]
s = Solver()
board = np.zeros((6,6))
for i in range(6) :
    for j in range(6) :
        board[i][j] = Int(column_indices[i]+row_indices[j])
        s.add(board[i][j]>0)
        s.add(board[i][j]<7)

for i in range(6) :
    for j in range(6) :
        for k in range(6) :
            s.add(board[i][j] != board[i][k])
            
for i in range(6) :
    for j in range(6) :
        for k in range(6) :
            s.add(board[k][i] != board[j][i])

for i in data :
    
    operator = i[1][-1]
    value = i[1][:len(i[i]) - 1]
    expression = []
    for j in i[0] :
        expression.append(str(board[column_indices.index(j[0])][row_indices.index(j[1])]))
        expression.append(operator)
    expression.pop()
    expression = " ".join(expression)
    expression += " == " + value
    
    s.add(eval(expression))
    
    

### Adding Constraints

### Printing the output

# Conclusions

The class room scheduling problem has been solved with the help of z3. In the assignment of the timetable given by the Z3 SAT solver, all the constraints are being satisfied. There are no evidences in the time table where constraints are violated. So, the above code can be used to schedule the classes at IIT Goa provided that proper input is given to it.